In [ ]:
# seq2seq을 이용한 기계번역기 구현

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [2]:
import urllib3
import zipfile
import shutil
import os
import pandas as pd
import numpy as np
# 모듈들 임포트 

In [3]:
urllib = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)

with urllib.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
  shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)
# 프랑스 -영어 병렬 코퍼스인 fra-eng.zip파일을 사용

In [4]:
lines = pd.read_csv('fra.txt', names=['src', 'tar','CC'], sep='\t')
lines.head(3)
# csv파일을 읽어와서 column 이름을 지정해줌 

,src,tar,CC
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [ ]:
lines = lines.loc[:, 'src':'tar']
lines.info()
# 모든행의 src와 tar 지정

In [ ]:
lines = lines[0:10000]
lines.sample(10)
# 만개의 샘플만 지정 

In [ ]:
lines.tar

0                    Va !
1                 Marche.
2                 Bouge !
3                 Salut !
4                  Salut.
              ...        
9995    Il était patient.
9996      Il fut parfait.
9997    Il a été parfait.
9998    Il était parfait.
9999     Il fut paralysé.
Name: tar, Length: 10000, dtype: object

In [ ]:
lines.tar = lines.tar.apply(lambda x: '\t ' + x + ' \n')
lines.sample(10)
# <sos>와 <eos> 대신 \t를 시작심볼, \n을 종료심볼로 추가 

In [ ]:
src_vocab = set() #문자의 집합을 구축

for line in lines.src: #1줄씩 읽고
  for char in line: #1개 문자씩 읽음
    src_vocab.add(char)

In [ ]:
src_vocab

In [ ]:
tar_vocab=set() #문자의 집합을 구축

for line in lines.tar: #1줄씩 읽고
  for char in line: #1개 문자씩 읽음
    tar_vocab.add(char)

In [ ]:
tar_vocab

In [ ]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1 

print(src_vocab_size)
print(tar_vocab_size)

# 문자 집합의 크기

72
93


In [ ]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

# 정렬해서 순서를 정해줌 

In [ ]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
# 정렬한 문자에 인덱스를 부여
print(tar_to_index)

{'\t': 1, '\n': 2, ' ': 3, '!': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '2': 15, '3': 16, '5': 17, '8': 18, '9': 19, ':': 20, '?': 21, 'A': 22, 'B': 23, 'C': 24, 'D': 25, 'E': 26, 'F': 27, 'G': 28, 'H': 29, 'I': 30, 'J': 31, 'K': 32, 'L': 33, 'M': 34, 'N': 35, 'O': 36, 'P': 37, 'Q': 38, 'R': 39, 'S': 40, 'T': 41, 'U': 42, 'V': 43, 'Y': 44, 'a': 45, 'b': 46, 'c': 47, 'd': 48, 'e': 49, 'f': 50, 'g': 51, 'h': 52, 'i': 53, 'j': 54, 'k': 55, 'l': 56, 'm': 57, 'n': 58, 'o': 59, 'p': 60, 'q': 61, 'r': 62, 's': 63, 't': 64, 'u': 65, 'v': 66, 'w': 67, 'x': 68, 'y': 69, 'z': 70, '\xa0': 71, '«': 72, '»': 73, 'À': 74, 'Ç': 75, 'É': 76, 'Ê': 77, 'à': 78, 'â': 79, 'ç': 80, 'è': 81, 'é': 82, 'ê': 83, 'î': 84, 'ï': 85, 'ô': 86, 'ù': 87, 'û': 88, 'œ': 89, '\u2009': 90, '’': 91, '\u202f': 92}


In [ ]:
print(src_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '5': 15, '7': 16, '8': 17, '9': 18, ':': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'Y': 44, 'a': 45, 'b': 46, 'c': 47, 'd': 48, 'e': 49, 'f': 50, 'g': 51, 'h': 52, 'i': 53, 'j': 54, 'k': 55, 'l': 56, 'm': 57, 'n': 58, 'o': 59, 'p': 60, 'q': 61, 'r': 62, 's': 63, 't': 64, 'u': 65, 'v': 66, 'w': 67, 'x': 68, 'y': 69, 'z': 70, 'é': 71}


In [ ]:
# 인코더의 입력이 될 영어문장 샘플 정수 인코딩 수행  
encoder_input = []

# 1개의 문장
for line in lines.src:
  temp_x = []
# 각 줄에서 한개의 char 
  for w in line:
        # 각 char를 정수로 변환
    temp_x.append(src_to_index[w])
  encoder_input.append(temp_x)

encoder_input

In [ ]:
encoder_input

In [ ]:
# 프랑스어 데이터 정수 인코딩 수행 
decoder_input = []

# 1개의 문장
for line in lines.tar:
  temp_x = []
# 각 줄에서 한개의 char 
  for w in line:
          # 각 char를 정수로 변환
    temp_x.append(tar_to_index[w])
  decoder_input.append(temp_x)

decoder_input

In [ ]:
print(decoder_input[:5])

[[1, 3, 43, 45, 3, 4, 3, 2], [1, 3, 34, 45, 62, 47, 52, 49, 12, 3, 2], [1, 3, 23, 59, 65, 51, 49, 3, 4, 3, 2], [1, 3, 40, 45, 56, 65, 64, 3, 4, 3, 2], [1, 3, 40, 45, 56, 65, 64, 12, 3, 2]]


In [ ]:
# 디코더의 예측값과 비교하기 위한 실제값 
decoder_target = []

for line in lines.tar:
  t = 0
  temp_x = []
  for w in line:
    if t>0: # 실제값엔 필요없는 \t제거함 
      temp_x.append(tar_to_index[w])
    t=t+1
  decoder_target.append(temp_x)

In [ ]:
print(decoder_target[:10]) # 인덱스 1제거된것 확인(\t)

[[3, 43, 45, 3, 4, 3, 2], [3, 34, 45, 62, 47, 52, 49, 12, 3, 2], [3, 23, 59, 65, 51, 49, 3, 4, 3, 2], [3, 40, 45, 56, 65, 64, 3, 4, 3, 2], [3, 40, 45, 56, 65, 64, 12, 3, 2], [3, 24, 59, 65, 62, 63, 92, 4, 3, 2], [3, 24, 59, 65, 62, 49, 70, 92, 4, 3, 2], [3, 37, 62, 49, 58, 49, 70, 3, 66, 59, 63, 3, 54, 45, 57, 46, 49, 63, 3, 78, 3, 66, 59, 63, 3, 47, 59, 65, 63, 3, 4, 3, 2], [3, 27, 53, 56, 49, 3, 4, 3, 2], [3, 27, 53, 56, 49, 70, 3, 4, 3, 2]]


In [ ]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

# 패딩을 하기위해 최대길이 확인 

In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
# 영어데이터샘플 패딩
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')
# 프랑스어 샘플 패딩 

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
# 원핫 인코딩 수행 

In [ ]:
# 훈련시키기 

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True) #인코더의 내부상태를 디코더에 넘겨주기위해

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태(두가지전달).
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.(컨텍스트벡터)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
# 프랑스어단어집합 크기만큼 뉴런을 배치 
decoder_outputs = decoder_softmax_layer(decoder_outputs) 
# 출력층에 소프트맥스 함수를 사용하여 실제값과 오차를 구함 

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)
# 입력으로 인코더,디코더, 디코더의 실제값

In [ ]:
# 번역기 동작 
# 1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻음
# 2. 상태와 <sos>에 해당하는 \t를 디코더로 보냄
# 3. 디코더가 <eos>에 해당하는 \n이 나올때까지 다음 문자를 예측하는 행동을 반복
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.(교사강요)
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [1]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())
# 인덱스로 부터 단어를 얻음

NameError: name 'src_to_index' is not defined

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

In [ ]:
-----------------------------------
입력 문장: Hi.
정답 문장: Salut ! 
번역 문장: Salut. 
-----------------------------------
입력 문장: I see.
정답 문장: Aha. 
번역 문장: Je change. 
-----------------------------------
입력 문장: Hug me.
정답 문장: Serrez-moi dans vos bras ! 
번역 문장: Serre-moi dans vos patents ! 
-----------------------------------
입력 문장: Help me.
정답 문장: Aidez-moi. 
번역 문장: Aidez-moi. 
-----------------------------------
입력 문장: I beg you.
정답 문장: Je vous en prie. 
번역 문장: Je vous en prie. 